In [1]:
import tensorflow as tf
import os

In [2]:
tf.app.flags.DEFINE_integer("max_step", 100, "模型训练步数")
tf.app.flags.DEFINE_string("model_dir", "./ckpt/linear_regression/model",  "模型文件的加载路径")
# Fix UnrecognizedFlagError: Unknown command line flag 'f'
tf.app.flags.DEFINE_string("f", "", "kernel")

FLAGS = tf.app.flags.FLAGS

In [3]:
def linear_regression():
    """
    实现一个线性回归预测
    一个神经元
    """
    with tf.variable_scope("data"):
        #1. 准备数据，x 特征值 [100, 1], y_true 目标值 [100, 1]
        x = tf.random_normal([100,1], mean=1.75, stddev=1.0,name="x_data")
        
        # y = 0.7 *x + 0.8
        # 矩阵相乘必须是二维的 [100,1] * [1,1] = [100, 1]
        y_true = tf.matmul(x, [[0.7]]) + 0.8
        
    with tf.variable_scope("model"):
        # 2. 建立线性回归模型 1个特征，1个权重，1个偏置 y = wx + b
        # 随机初始化一个权重和偏置值，让他去计算损失，然后在当前状态下优化
        # 用变量定义才能优化
        # tainable参数：
        weight = tf.Variable(tf.random_normal([1,1],mean=0.0, stddev=1.0), name="w")
        bias = tf.Variable(0.0, name="b")
        
        y_predict = tf.matmul(x, weight) + bias
        
    with tf.variable_scope("loss"):
        # 3. 建立损失函数，均方误差
        loss = tf.reduce_mean(tf.square(y_predict - y_true))
        
    with tf.variable_scope("optimizer"):
        train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    # 收集 tensor
    tf.summary.scalar("losses", loss)
    tf.summary.histogram("weight", weight)
    
    # 定义合并tensor的op
    merged = tf.summary.merge_all()
    
    # 定义一个初始化变量的op
    init_op = tf.global_variables_initializer()
    
    # 定义一个保持模型的实例
    saver = tf.train.Saver()
    
    #通过会话运行程序
    with tf.Session() as sess:
        #初始化变量
        sess.run(init_op)
        
        print("随机初始化的参数权重：%f, 偏置：%f" % (weight.eval(), bias.eval()))
        
        # 建立tensorboard事件文件
        filewriter = tf.summary.FileWriter("/tf/logs/linear_regression/", graph=sess.graph)
        
        #加载模型，覆盖模型中随机定义的参数，从上次训练的参数开始
        if os.path.exists(FLAGS.model_dir):
            saver.restore(sess, FLAGS.model_dir)
            print("restore model")
        
        #循环训练，运行优化
        for i in range(FLAGS.max_step):
            sess.run(train_op)
            
            # 运行合并的tensor
            summary = sess.run(merged)
            filewriter.add_summary(summary, i)
            
            print("第%d次优化的权重：%f, 偏置:%f, loss:%f" % (i, weight.eval(), bias.eval(), loss.eval()))
            
            
        saver.save(sess, FLAGS.model_dir)
        
    return None

In [4]:
if __name__ == "__main__":
    linear_regression()

随机初始化的参数权重：0.809342, 偏置：0.000000
第0次优化的权重：1.013875, 偏置:0.118971, loss:0.101936
第1次优化的权重：1.020659, 偏置:0.147317, loss:0.136145
第2次优化的权重：0.999272, 偏置:0.171854, loss:0.111379
第3次优化的权重：0.997707, 偏置:0.202383, loss:0.099887
第4次优化的权重：0.966964, 偏置:0.221517, loss:0.117049
第5次优化的权重：0.947348, 偏置:0.240780, loss:0.091781
第6次优化的权重：0.955095, 偏置:0.271906, loss:0.077742
第7次优化的权重：0.931953, 偏置:0.288375, loss:0.054619
第8次优化的权重：0.934857, 偏置:0.320636, loss:0.039692
第9次优化的权重：0.919536, 偏置:0.341167, loss:0.061692
第10次优化的权重：0.898306, 偏置:0.349544, loss:0.046634
第11次优化的权重：0.894539, 偏置:0.368691, loss:0.055134
第12次优化的权重：0.876538, 偏置:0.382972, loss:0.035131
第13次优化的权重：0.880897, 偏置:0.406138, loss:0.040248
第14次优化的权重：0.863597, 偏置:0.417491, loss:0.031730
第15次优化的权重：0.865090, 偏置:0.440102, loss:0.026764
第16次优化的权重：0.864361, 偏置:0.458616, loss:0.033154
第17次优化的权重：0.855318, 偏置:0.471701, loss:0.022417
第18次优化的权重：0.840984, 偏置:0.482636, loss:0.028251
第19次优化的权重：0.827303, 偏置:0.493728, loss:0.023536
第20次优化的权重：0.829242, 偏置:0.512491, loss